In [2]:
!pip install langchain langchain_community # popular framework for generative ai
%pip install --upgrade --quiet huggingface_hub
!pip install faiss-cpu # vectorstore
!pip install pypdf # loader in rag
!pip install langchain_huggingface
!pip install chromadb # vectorstore
!pip install langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency

## Let's import necessary libraries

In [23]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_tkn = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN") # for accessing my secret HuggingFace token

from langchain.llms import HuggingFaceHub 
from langchain_huggingface import HuggingFaceEndpoint # for accessing huggingface models
from langchain_huggingface import HuggingFaceEmbeddings # embeding the documents in the vectorstore
from langchain_huggingface import ChatHuggingFace # chat model
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS,Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage,AIMessage

## Document loading

In [4]:
pdfloader = PyPDFLoader('/kaggle/input/mobile/Mobile Computing Notes -All (4).pdf')
document = pdfloader.load()

## Splitting the document into chunks

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts = splitter.split_documents(document)
# RecursiveCharacterTextSplitting in Langchain is a technique for splitting text into smaller chunks based on character boundaries.
# It employs a recursive approach, meaning it repeatedly breaks down the text until it reaches the desired chunk size

# Chunk overlap is the number of characters that should overlap between two adjacent chunks.
# The chunk size and chunk overlap parameters can be used to control the granularity of the text splitting.
# A smaller chunk size will result in more chunks, while a larger chunk size will result in fewer chunks.

## Creating an embeddings from the documents

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma.from_documents(texts,embedding=embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Let's now load of chat model from huggingface

In [7]:
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=hf_tkn
)

chat_model = ChatHuggingFace(llm=llm)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

## Let's create a memory system prompt for interacting with the 'chat_history'

In [25]:
memory_system_prompt = (
                     "Given a chat history and latest user question"
                     "which might reference context in the chat history"
                     "formulate a standalone question which can be understood"
)

memory_prompt = ChatPromptTemplate.from_messages([
    ('system',memory_system_prompt),
    MessagesPlaceholder('chat_history'),# allow us to pass a list of messages to the prompt using 'chat_history'
    ('human','{input}')
])

## creating retriever

In [14]:
retriever = db.as_retriever()

In [26]:
# this attaches chat_history to the retriever
history_aware_retriever = create_history_aware_retriever(chat_model,retriever,memory_prompt)

## Now let's create our normal system prompt

In [16]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [27]:
# create_stuff_documents_chain is used to generate a QA chain with input keys "content,chat_history,input"
question_answer_chain = create_stuff_documents_chain(chat_model, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

## Let's now define a sample user input

In [28]:
def generate_response(chain,user_inp,chat_hist):
    response = chain.invoke({
        "input": user_inp,
        "chat_history": chat_hist
                            })
    return response['answer']
chat_history = []
while True:
    user_input = input('You:')
    if user_input == 'Exit':
        break
    response = generate_response(rag_chain,user_input,chat_history)
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))

    print('Assistant:',response)

You: Define mobile computing


Assistant: Mobile computing refers to the ability to access network services anywhere, at any time, and from any location, using portable devices such as smartphones, tablets, or laptops with wireless connectivity. It enables almost unrestricted mobility, as users no longer need to be in a fixed position to access network resources. Mobile computing encompasses technologies that enable communication, data storage, and processing while on the move, such as Bluetooth, cellular networks, and GPS (Global Positioning System


You: Examples of portable devices it uses


Assistant: Some examples of portable devices used in mobile computing include:

1. Smartphones: These are handheld devices that combine the functionality of a mobile phone, computer, and other wireless devices such as a GPS navigation system, MP3 player, and digital camera.

2. Tablets: These are smaller, portable, and lighter versions of laptops. They have a touchscreen display, run on mobile operating systems, and typically do not have a physical keyboard.

3. Net


You: Exit
